<a href="https://colab.research.google.com/github/PencilMario/stable-diffusion-webui/blob/master/StableDiffusionUI_(adapted_to_NovelAILeaks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StableDiffusionWebui 训练自动保存版
基于笔记本https://github.com/JingShing/novelai-colab-ver/blob/main/webui-ver/StableDiffusionUI_(adapted_to_NovelAILeaks).ipynb 修改而成

主要在跑训练时自动将训练结果保存至google drive, 如果你真的想用这个跑图也是可以的

#预处理

In [14]:
#@title 工具函数
#@markdown 在运行以下指令块前请先运行这个
import subprocess
def run_command_by_line(command_args):
    def print_pipe(raw):
        return print(raw.decode("utf-8"), end='')
    with subprocess.Popen(command_args, stdout=subprocess.PIPE, stderr=subprocess.PIPE) as process:
      while process.poll() is None:
        print_pipe(process.stdout.readline())
      [print_pipe(line) for line in process.stderr.readlines()]
    return

def check_select(str, instr):
  strs="qwertyuiopasdfghjklzxcvbnm"
  str=str.lower()
  result=[]
  for i in strs:
    if str.find(i) != -1:
      result.append(i)
  for a in result:
    if a==instr:
      return True
  return False



In [ ]:
#@title 挂载Google Drive
#@markdown **警告：可能会导致Google扫描你的网盘，建议在干净的网盘账号上使用，以防万一:)**
#@markdown > 如果你不需要训练风格模型，即只用来跑图，那么不需要挂载谷歌网盘，直接运行下一个语句块即可。

#@markdown 训练时的模型和预览图将被保存到谷歌网盘的`StableDiffusionTraining`文件夹

#@markdown *小提示：colab的根目录是/content*  
#@markdown *如：sample_data在程序内的目录是/content/sample_data*
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
#@title clone webui仓库
%cd /content/
!git clone https://github.com/PencilMario/stable-diffusion-webui
%cd stable-diffusion-webui

In [ ]:
#@title 从NovelAlLeaks下载模型
#@markdown 输入指定的字符将会下载对应模型
#@markdown
#@markdown a - *7G animefull-final-latest （可能不工作)*
#@markdown 
#@markdown b - *4G animefull-final-pruned*
#@markdown
#@markdown c - *4G animefull-final-pruned （备用链接 | 推荐）*
#@markdown
#@markdown d - *hpyernetworks 全局风格模型文件*
#@markdown

#@markdown > 如果什么都不填，相当于填入c, ab从谷歌网盘下载，当人数过多时可能无法下载 

下载模型文件 = "cd" #@param {type:"string"}
为每个模型下载权重文件 = True #@param {type:"boolean"}
!mkdir -p /content/stable-diffusion-webui/models/Stable-diffusion /content/stable-diffusion-webui/models/hypernetworks
%cd /content/stable-diffusion-webui/models/Stable-diffusion/
if check_select(下载模型文件, 'a'):
  !gdown 17WWd6KEsBj7D_0TyGp8aXHQDlchYVs1a -O /content/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
  if 为每个模型下载权重文件:
    !curl -Lo model.vae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 
if check_select(下载模型文件, 'b'):
  !gdown 1d3f2fvN2gLRocBahZrXe_v1EEHuqpUzT -O /content/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
  if 为每个模型下载权重文件:
    !curl -Lo model.vae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 

if check_select(下载模型文件, 'c'):
  !curl -Lo model.ckpt https://cloudflare-ipfs.com/ipfs/bafybeicpamreyp2bsocyk3hpxr7ixb2g2rnrequub3j2ahrkdxbvfbvjc4/model.ckpt
  if 为每个模型下载权重文件:
    !curl -Lo model.vae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 

if check_select(下载模型文件, 'd'):
  !curl -L https://cloudflare-ipfs.com/ipfs/bafybeiduanx2b3mcvxlwr66igcwnpfmk3nc3qgxlpwh6oq6m6pxii3f77e/_modules.tar | tar x -C /content/stable-diffusion-webui/models/hypernetworks

if check_select(下载模型文件, 'e'):
  !curl -L https://cloudflare-ipfs.com/ipfs/bafybeie3hdjchxs5tz4n75bos53nhcklslguxchdurc2ynrzcfv2kwyklu/embeddings.tar | tar x -C /content/stable-diffusion-webui/embeddings

if 下载模型文件 is None:
  run_command_by_line(['curl','-Lo','model.ckpt','https://cloudflare-ipfs.com/ipfs/bafybeicpamreyp2bsocyk3hpxr7ixb2g2rnrequub3j2ahrkdxbvfbvjc4/model.ckpt'])
  if 为每个模型下载权重文件:
    !curl -Lo model.vae.pt https://cloudflare-ipfs.com/ipfs/bafybeiccldswdd3wvg57jhclcq53lvsc6gizasiblwayvhlv6eq4wow7wu/animevae.pt 
print("下载完成")

#运行sd-webui


In [ ]:
#@markdown 启动webui。你将得到一个***.gradio.app的链接

#@markdown 启动项参数：

#@markdown `--share` - 创建在线gradio.app链接
#@markdown 
#@markdown `--gradio-debug` - 打印输出到控制台
#@markdown 
#@markdown `--gradio-auth me:qwerty` - 登录验证：用户名me，密码qwerty
#@markdown 
#@markdown `--deepdanbooru` - 支持使用deepdanbooru进行数据集预处理
%cd /content/stable-diffusion-webui
CMD = "COMMANDLINE_ARGS="
REQFILE="REQS_FILE="
#@markdown ----------
启动项 = "--share --gradio-debug --deepdanbooru" #@param{type:"string"}

依赖文件 = "requirements.txt" #@param {type:"string"}
RUN = "python launch.py"
run_command_by_line(['python', 'launch.py', CMD+启动项, REQFILE+依赖文件 ])
#!COMMANDLINE_ARGS="--share --gradio-debug --deepdanbooru" REQS_FILE="requirements.txt" python launch.py

#工具
建议挂载网盘后使用

In [ ]:
#@title 解压文件
#@markdown .zip文件路径  
filename = "/content/drive/MyDrive/sample.zip" #@param
#@markdown 解压至文件夹
outpath = "/content/extract_files" #@param

import os
os.makedirs(outpath,exist_ok=True)
#@markdown ~~没成功就再试一遍~~
run_command_by_line(["7z", 'x', filename, "-o"+outpath if outpath[-1]!="/" else "-o"+outpath+"/"])

In [ ]:
#@title 更新sd(用于测试)
%cd /content/stable-diffusion-webui
!git pull

In [24]:
#@title 删除sd（用于测试）
import shutil 
shutil.rmtree("/content/stable-diffusion-webui")

#杂项

*以下语句块只有在**#运行sd-webui**结束后才建议使用*

In [ ]:
#@title 保存生成的图片

!zip -r /content/stable-diffusion-webui /content/stable-diffusion-webui/outputs 

In [ ]:
#@title 保存图片到谷歌drive
#@markdown **警告：可能会导致Google扫描你的网盘，建议在干净的网盘账号上使用，以防万一:)**
from google.colab import drive # type: ignore

try:
   drive_path = "/content/drive"
   drive.mount(drive_path,force_remount=False)
except:
   print("...error mounting drive or with drive path variables")

!cp -r "/content/stable-diffusion-webui/outputs" "/content/drive/MyDrive"